<a href="https://colab.research.google.com/github/arcossci/agentic/blob/master/Hierarchical_Agent_Team_LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required packages in Colab
!pip install -qU langgraph langchain langchain_openai langchain_community tavily-python

  Using cached langgraph-0.3.27-py3-none-any.whl.metadata (7.7 kB)
  Using cached langchain_openai-0.3.12-py3-none-any.whl.metadata (2.3 kB)
  Using cached tavily_python-0.5.4-py3-none-any.whl.metadata (91 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.1 MB/s eta 0:00:00
  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [ ]:
# Import necessary libraries
import os
import getpass
from typing import Annotated, List, TypedDict
from langchain_openai import AzureChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, AIMessage
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import create_react_agent
from google.colab import userdata

# Set up environment variables for API keys in Colab
os.environ["AZURE_OPENAI_API_KEY"] = userdata.get('AZURE_INFERENCE_SDK_KEY')
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://wafer-m1tixc2n-eastus2.openai.azure.com/"
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

# Define Azure OpenAI GPT-4o-mini model
llm = AzureChatOpenAI(
    azure_deployment="gpt-4o-mini",  # Adjust based on your Azure deployment name
    api_version="2023-05-15",        # Use the appropriate API version
    temperature=0
)

# Define tools
tavily_tool = TavilySearchResults(max_results=5)

@tool
def write_document(content: str, filename: str) -> str:
    """Writes content to a file and returns a confirmation message."""
    with open(filename, "w") as f:
        f.write(content)
    return f"Document written to {filename}"

# Define state for the overall graph
class OverallState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "A list of messages"]
    team_members: List[str]
    next: str
    instructions: str

# Define state for the research team
class ResearchTeamState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "A list of messages"]
    team_members: List[str]
    instructions: str

# Define state for the writing team
class WritingTeamState(TypedDict):
    messages: Annotated[List[HumanMessage | AIMessage], "A list of messages"]
    team_members: List[str]
    instructions: str
    current_files: str

# Research team nodes
search_agent = create_react_agent(llm, tools=[tavily_tool])
def search_node(state: ResearchTeamState) -> dict:
    result = search_agent.invoke({"messages": state["messages"]})
    return {
        "messages": state["messages"] + [AIMessage(content=result["messages"][-1].content, name="search")]
    }

# Writing team nodes
doc_writing_agent = create_react_agent(llm, tools=[write_document])
def doc_writing_node(state: WritingTeamState) -> dict:
    result = doc_writing_agent.invoke({"messages": state["messages"]})
    return {
        "messages": state["messages"] + [AIMessage(content=result["messages"][-1].content, name="doc_writer")]
    }

# Supervisor node
def supervisor_node(state: OverallState) -> dict:
    messages = state["messages"]
    last_message = messages[-1].content.lower()
    if "research" in last_message:
        return {"next": "research_team"}
    elif "write" in last_message or "report" in last_message:
        return {"next": "writing_team"}
    else:
        return {"next": "END"}

# Build the research team subgraph
research_graph = StateGraph(ResearchTeamState)
research_graph.add_node("search", search_node)
research_graph.set_entry_point("search")
research_graph.set_finish_point("search")  # Research completes after search

# Build the writing team subgraph
writing_graph = StateGraph(WritingTeamState)
writing_graph.add_node("doc_writer", doc_writing_node)
writing_graph.set_entry_point("doc_writer")
writing_graph.set_finish_point("doc_writer")  # Writing completes after doc_writer

# Build the main graph
graph = StateGraph(OverallState)
graph.add_node("supervisor", supervisor_node)
graph.add_node("research_team", research_graph.compile())
graph.add_node("writing_team", writing_graph.compile())
graph.add_conditional_edges(
    "supervisor",
    lambda state: state["next"],
    {
        "research_team": "research_team",
        "writing_team": "writing_team",
        "END": END
    }
)
graph.add_edge("research_team", "supervisor")
graph.add_edge("writing_team", "supervisor")
graph.set_entry_point("supervisor")

# Compile the graph
research_chain = graph.compile()

# Function to run the graph with streaming
def run_research_chain(query: str):
    initial_state = {
        "messages": [HumanMessage(content=query)],
        "team_members": ["search", "doc_writer"],
        "next": "supervisor",
        "instructions": "Research AI trends and write a report."
    }
    for s in research_chain.stream(initial_state, {"recursion_limit": 100}):
        print(s)
        print("----")

# Test the chain
query = "Research the latest AI trends in 2025 and write a report."
run_research_chain(query)

{'supervisor': {'next': 'research_team'}}
----
{'research_team': {'messages': [HumanMessage(content='Research the latest AI trends in 2025 and write a report.', additional_kwargs={}, response_metadata={}, id='5c367fa7-f86e-442f-82a3-ab5a9e5dc34e'), AIMessage(content="### Report on Latest AI Trends in 2025\n\nAs we look ahead to 2025, the landscape of artificial intelligence (AI) is set to undergo significant transformations. The following report outlines the key trends that are expected to shape the future of AI, based on recent analyses and expert predictions.\n\n#### 1. **Edge AI Dominance**\nEdge AI is anticipated to become a major trend in 2025, moving data processing closer to the source of data generation. This shift will enhance speed, security, and responsiveness, making it ideal for applications in self-driving cars, smart appliances, and wearable technology. The demand for real-time applications will drive the adoption of edge AI, allowing for more efficient and secure data h

KeyboardInterrupt: 

In [ ]:
Create Tools